# Lab4-Assignment about Named Entity Recognition and Classification

This notebook describes the assignment of Lab 4 of the text mining course. We assume you have succesfully completed Lab1, Lab2 and Lab3 as welll. Especially Lab2 is important for completing this assignment.

**Learning goals**
* going from linguistic input format to representing it in a feature space
* working with pretrained word embeddings
* train a supervised classifier (SVM)
* evaluate a supervised classifier (SVM)
* learn how to interpret the system output and the evaluation results
* be able to propose future improvements based on the observed results


## Credits
This notebook was originally created by [Marten Postma](https://martenpostma.github.io) and [Filip Ilievski](http://ilievski.nl) and adapted by Piek vossen

## [Points: 18] Exercise 1 (NERC): Training and evaluating an SVM using CoNLL-2003

**[4 point] a) Load the CoNLL-2003 training data using the *ConllCorpusReader* and create for both *train.txt* and *test.txt*:**

    [2 points]  -a list of dictionaries representing the features for each training instances, e..g,
    ```
    [
    {'words': 'EU', 'pos': 'NNP'}, 
    {'words': 'rejects', 'pos': 'VBZ'},
    ...
    ]
    ```

    [2 points] -the NERC labels associated with each training instance, e.g.,
    dictionaries, e.g.,
    ```
    [
    'B-ORG', 
    'O',
    ....
    ]
    ```

In [32]:
filename = "/Users/owhy/Desktop/github/TextMining-VU-2024/lab4/CONLL2003"

In [33]:
from nltk.corpus.reader import ConllCorpusReader
### Adapt the path to point to the CONLL2003 folder on your local machine
train = ConllCorpusReader(filename, "train.txt", ["words", "pos", "ignore", "chunk"])
training_features = []
training_gold_labels = []

for token, pos, ne_label in train.iob_words():
    a_dict = {
        "word": token,
        "pos": pos,
        "word.lower()": token.lower(),  # lower case variant of the token
        "suffix-2char": token[-2:],  # suffix of 2 characters
        "capitalization": token.isupper(),  # initial captial
        "capitalization_all": token.istitle(),  # all words ini caps
        "digit": token.isdigit(),
    }
    training_features.append(a_dict)
    training_gold_labels.append(ne_label)

In [36]:
### Adapt the path to point to the CONLL2003 folder on your local machine
test = ConllCorpusReader(filename, "test.txt", ["words", "pos", "ignore", "chunk"])

test_features = []
test_gold_labels = []
for token, pos, ne_label in test.iob_words():
    a_dict = {
        "word": token,
        "pos": pos,
        "word.lower()": token.lower(),  # lower case variant of the token
        "suffix-2char": token[-2:],  # suffix of 2 characters
        "capitalization": token.isupper(),  # initial captial
        "capitalization_all": token.istitle(),  # all words ini caps
        "digit": token.isdigit(),
    }
    test_features.append(a_dict)
    test_gold_labels.append(ne_label)

In [38]:
# Print examples from the training data
print("Training Data Examples:")
for i in range(2):
    print("Features:", training_features[i])
    print("Gold Label:", training_gold_labels[i])
    print()

# Print examples from the test data
print("Test Data Examples:")
for i in range(2):  # Iterate over the entire test_features list
    print("Features:", test_features[i])
    print("Gold Label:", test_gold_labels[i])
    print()

Training Data Examples:
Features: {'word': 'EU', 'pos': 'NNP', 'word.lower()': 'eu', 'suffix-2char': 'EU', 'capitalization': True, 'capitalization_all': False, 'digit': False}
Gold Label: B-ORG

Features: {'word': 'rejects', 'pos': 'VBZ', 'word.lower()': 'rejects', 'suffix-2char': 'ts', 'capitalization': False, 'capitalization_all': False, 'digit': False}
Gold Label: O

Test Data Examples:
Features: {'word': 'SOCCER', 'pos': 'NN', 'word.lower()': 'soccer', 'suffix-2char': 'ER', 'capitalization': True, 'capitalization_all': False, 'digit': False}
Gold Label: O

Features: {'word': '-', 'pos': ':', 'word.lower()': '-', 'suffix-2char': '-', 'capitalization': False, 'capitalization_all': False, 'digit': False}
Gold Label: O



**[2 points] b) provide descriptive statistics about the training and test data:**
* How many instances are in train and test?
* Provide a frequency distribution of the NERC labels, i.e., how many times does each NERC label occur?
* Discuss to what extent the training and test data is balanced (equal amount of instances for each NERC label) and to what extent the training and test data differ?

Tip: you can use the following `Counter` functionality to generate frequency list of a list:

In [40]:
from collections import Counter

# Count the occurrences of each NERC label in the training data
train_label_counts = Counter(training_gold_labels)

# Count the occurrences of each NERC label in the test data
test_label_counts = Counter(test_gold_labels)

# Print the number of instances in train and test
print("Number of instances in train:", len(training_gold_labels))
print("Number of instances in test:", len(test_gold_labels))

# Print the frequency distribution of NERC labels in train and test
print("\nFrequency distribution of NERC labels in train:")
print(train_label_counts)

print("\nFrequency distribution of NERC labels in test:")
print(test_label_counts)

# Discuss the balance and differences between train and test data
train_total_instances = len(training_gold_labels)
test_total_instances = len(test_gold_labels)

train_balance = {
    label: count / train_total_instances for label, count in train_label_counts.items()
}
test_balance = {
    label: count / test_total_instances for label, count in test_label_counts.items()
}

print("\nBalance of NERC labels in train data:")
print(train_balance)

print("\nBalance of NERC labels in test data:")
print(test_balance)

Number of instances in train: 203621
Number of instances in test: 46435

Frequency distribution of NERC labels in train:
Counter({'O': 169578, 'B-LOC': 7140, 'B-PER': 6600, 'B-ORG': 6321, 'I-PER': 4528, 'I-ORG': 3704, 'B-MISC': 3438, 'I-LOC': 1157, 'I-MISC': 1155})

Frequency distribution of NERC labels in test:
Counter({'O': 38323, 'B-LOC': 1668, 'B-ORG': 1661, 'B-PER': 1617, 'I-PER': 1156, 'I-ORG': 835, 'B-MISC': 702, 'I-LOC': 257, 'I-MISC': 216})

Balance of NERC labels in train data:
{'B-ORG': 0.03104296708099852, 'O': 0.832811939829389, 'B-MISC': 0.016884309575142052, 'B-PER': 0.0324131597428556, 'I-PER': 0.022237392017522752, 'B-LOC': 0.03506514553999833, 'I-ORG': 0.018190658134475325, 'I-MISC': 0.00567230295499973, 'I-LOC': 0.005682125124618777}

Balance of NERC labels in test data:
{'O': 0.8253041886508022, 'B-LOC': 0.03592118014428771, 'B-PER': 0.03482287067944438, 'I-PER': 0.0248950145364488, 'I-LOC': 0.005534618283622268, 'B-MISC': 0.015117906751372886, 'I-MISC': 0.004651663

**[2 points] c) Concatenate the train and test features (the list of dictionaries) into one list. Load it using the *DictVectorizer*. Afterwards, split it back to training and test.**

Tip: You’ve concatenated train and test into one list and then you’ve applied the DictVectorizer.
The order of the rows is maintained. You can hence use an index (number of training instances) to split the_array back into train and test. Do NOT use: `
from sklearn.model_selection import train_test_split` here.


In [5]:
from sklearn.feature_extraction import DictVectorizer

In [6]:
vec = DictVectorizer()
the_array = # your code here

SyntaxError: invalid syntax (1177049845.py, line 2)

**[4 points] d) Train the SVM using the train features and labels and evaluate on the test data. Provide a classification report (sklearn.metrics.classification_report).**
The train (*lin_clf.fit*) might take a while. On my computer, it took 1min 53s, which is acceptable. Training models normally takes much longer. If it takes more than 5 minutes, you can use a subset for training. Describe the results:
* Which NERC labels does the classifier perform well on? Why do you think this is the case?
* Which NERC labels does the classifier perform poorly on? Why do you think this is the case?

In [ ]:
from sklearn import svm

In [ ]:
lin_clf = svm.LinearSVC()

In [ ]:
##### [ YOUR CODE SHOULD GO HERE ]
# lin_clf.fit( # your code here

**[6 points] e) Train a model that uses the embeddings of these words as inputs. Test again on the same data as in 2d. Generate a classification report and compare the results with the classifier you built in 2d.**

In [ ]:
# your code here

# --------------------

# --------------------

## [Points: 10] Exercise 2 (NERC): feature inspection using the [Annotated Corpus for Named Entity Recognition](https://www.kaggle.com/abhinavwalia95/entity-annotated-corpus)
**[6 points] a. Perform the same steps as in the previous exercise. Make sure you end up for both the training part (*df_train*) and the test part (*df_test*) with:**
* the features representation using **DictVectorizer**
* the NERC labels in a list

Please note that this is the same setup as in the previous exercise:
* load both train and test using:
    * list of dictionaries for features
    * list of NERC labels
* combine train and test features in a list and represent them using one hot encoding
* train using the training features and NERC labels

In [ ]:
import pandas

In [ ]:
##### Adapt the path to point to your local copy of NERC_datasets
path = '/Users/piek/Desktop/ONDERWIJS/data/nerc_datasets/kaggle/ner_v2.csv'
kaggle_dataset = pandas.read_csv(path, error_bad_lines=False)

In [ ]:
len(kaggle_dataset)

In [ ]:
df_train = kaggle_dataset[:100000]
df_test = kaggle_dataset[100000:120000]
print(len(df_train), len(df_test))

**[4 points] b. Train and evaluate the model and provide the classification report:**
* use the SVM to predict NERC labels on the test data
* evaluate the performance of the SVM on the test data

Analyze the performance per NERC label.

## End of this notebook